# PyTorch básico 2: montando e treinando um modelo

Saindo de exemplos de regressão e indo para classificação vamos olhar agora para um dataset que tenta trazer o problema de veículos autônomos para um ambiente mais simples.

##  Classificação de imagens


Temos um pequeno robô andando numa pista feita de papel. O robô tem uma câmera frontal possibilitando que ele veja para onde ele está indo. Assim, dada uma imagem ele precisa saber que ação ele vai tomar: $\uparrow, \leftarrow, \rightarrow$. Queremos que o robô aprenda a andar na pista sozinho. 

Nós podemos tentar resolver esse problema como um problema de **aprendizado supervisionado** temos um dataset $(\mathbf{x}_{1}, y_{1}), \dots ,(\mathbf{x}_{N}, y_{N})$ onde $\mathbf{x}_i \in \mathbb{R}^{45\times 80 \times 3}$ é uma imagem e $y_i \in \{ \uparrow, \leftarrow, \rightarrow \}$ é um rótulo. Assim o problema de direção autônoma se resume em aprender uma função $f$ que associa para cada imagem uma distribuição de probabilidade sobre os rótulos.

<table>
<tr>
<td>
<img align="middle"   width='600' heith='100' src='images/image_classification.png'>
</td>
</tr>
</table>

Mais informações sobre esse problema você pode encontrar [aqui](https://medium.com/@project_m/self-drives-me-crazy-from-0-to-self-driving-car-in-150-hours-bf4f68d50d8a).


In [ ]:
# notebook feito para a versão 0.4.0 do Pytorch 

import os
import subprocess
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from plots import plot9images, plot_confusion_matrix, plot_histogram_from_labels
from util import randomize_in_place
% matplotlib inline
print("PyTorch version = {} ".format(torch.__version__))

Em primeiro lugar vamos ter que pegar os dados

In [ ]:
# Essa célula pode demorar de acordo com sua conexão de internet.
# Olhe o terminal para mais informações sobre o download
if not os.path.exists("self_driving_pi_car_data"):
    pro = subprocess.Popen(["bash", "download.sh"])
    pro.wait()

Nesse notebook vamos usar apenas um pedaço dos dados.

In [ ]:
raw_X = np.load("self_driving_pi_car_data/train_data.npy")
raw_y = np.load("self_driving_pi_car_data/train_labels.npy")
randomize_in_place(raw_X, raw_y)
valid_X = raw_X[0:1000]
valid_y = raw_y[0:1000]
test_X = raw_X[1000:2000]
test_y = raw_y[1000:2000]
train_X = raw_X[2000:]
train_y = raw_y[2000:]

In [ ]:
command2int = {"forward": 0, "left": 1, "right": 2}
int2command = {i[1]: i[0] for i in command2int.items()}

print("Informações sobre os dados\n")
print("- Número de dados de treinamento = {}\n".format(train_X.shape[0]))
print("- Número de dados de validação = {}\n".format(valid_X.shape[0]))
print("- Número de dados de teste = {}\n".format(test_X.shape[0]))
print("- Número de features = {}\n".format(test_X.shape[1]))
print("- Número de classes = {}\n".format(3))
print("- Legenda das classes:  (0 = forward, 1 = left, 2 = right)\n")

### Olhando o dataset

Esses dados foram coletados fazendo com que o robô desse voltas nessa pista:

<table>
<tr>
<td>
<img align="middle" width='600' heith='100' src='images/train_track.png'>
</td>
</tr>
</table>

A cada ação tomada o robô tirava uma foto 45x80x3 (altura, largura, número de canais) e associava a ação tomada a essa foto.

As observações $\mathbf{x}_{1}, \dots, \mathbf{x}_{N}$ são as imagens transformadas em vetores (pegamos a matrix $45\times80\times3$ e a achatamos num vetor de tamanho $10800 = 45^*80^*3$). Cada feature aqui é a intensidade de um pixel da imagem (variando de 0 a 255).


Podemos olhar alguns exemplos: 

In [ ]:
img9 = train_X[0:9]
labels9 = train_y[0:9]
labels9 = [int2command[i] for i in labels9]
img9 = img9.reshape((9, 45, 80, 3)) 
img9 = img9[...,::-1]
plot9images(img9, labels9, (45, 80, 3))

Também podemos olhar como estão as distribuição das classes nos diferentes conjuntos de dados

In [ ]:
labels_legend = ["forward", "left", "right"]
plot_histogram_from_labels(train_y, labels_legend, "train data")
plot_histogram_from_labels(valid_y, labels_legend, "valid data")
plot_histogram_from_labels(test_y, labels_legend, "test data")

##  Organizando o código

Em primeiro lugar vamos criar uma classe para guardar todos os hiper parâmetros.

In [ ]:
class LRConfig(object):
    """
    Holds logistic regression model hyperparams.
    
    :param height: image height
    :type heights: int
    :param width: image width
    :type width: int
    :param channels: image channels
    :type channels: int
    :param batch_size: batch size for training
    :type batch_size: int
    :param epochs: number of epochs
    :type epochs: int
    :param save_step: when step % save_step == 0, the model
                      parameters are saved.
    :type save_step: int
    :param learning_rate: learning rate for the optimizer
    :type learning_rate: float
    :param momentum: momentum param
    :type momentum: float
    """
    def __init__(self,
                 height=45,
                 width=80,
                 channels=3,
                 classes=3,
                 batch_size=32,
                 epochs=3,
                 save_step=100,
                 learning_rate=0.01,
                 momentum=0.1):
        self.height = height
        self.width = width
        self.classes = classes
        self.channels = channels
        self.batch_size = batch_size
        self.epochs = epochs
        self.save_step = save_step
        self.learning_rate = learning_rate
        self.momentum = momentum
        

    def __str__(self):
        """
        Get all attributs values.
        :return: all hyperparams as a string
        :rtype: str
        """
        status = "height = {}\n".format(self.height)
        status += "width = {}\n".format(self.width)
        status += "channels = {}\n".format(self.channels)
        status += "classes = {}\n".format(self.classes)
        status += "batch_size = {}\n".format(self.batch_size)
        status += "epochs = {}\n".format(self.epochs)
        status += "save_step = {}\n".format(self.save_step)
        status += "learning_rate = {}\n".format(self.learning_rate)
        status += "momentum = {}\n".format(self.momentum)
        return status


In [ ]:
lr_config = LRConfig()
print("Os hiper parâmetros do modelo de regressão logística são:\n")
print(lr_config)

Vamos transformar as imagens em tensores e vamos usar a classe `TensorDataset` para guardar os dados.

Ao contrário do enunciado original, que copia os tensores para ter tanto a versão com bytes como a versão com floats, aqui eu apenas fico com as coisas que já foram lidas.

In [ ]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(torch.from_numpy(train_X),
                              torch.from_numpy(train_y))
valid_dataset = TensorDataset(torch.from_numpy(valid_X),
                              torch.from_numpy(valid_y))
test_dataset = TensorDataset(torch.from_numpy(test_X),
                             torch.from_numpy(test_y))

Para amostrar os dados vamos usar a classe `DataLoader` e vamos definir a classe `DataHolderGentle` para exclusivamente lidar com os dados.

A classe `GentleLoader` é apenas um wrapper em cima da classe `DataLoader` garantindo que os tensores só sejam convertidos para float (o que o torna 4 vezes maior) e para long (o que o torna 8 vezes maior) quando estes forem ser utilizados pelo modelo.

Se trata, então, de uma troca, que preserva memória mas talvez deixe o treinamento mais devagar, pois pra cada vez que se acessa um batch será necessário transformar os tipos dos tensores.

Caso você não tenha problema de memória, você pode importar a clase `DataHolder` do arquivo DataHolder.py.

In [ ]:
from torch.utils.data import DataLoader

class GentleLoader:
    def __init__(self, dataset, batch_size, shuffle):
        self.dataLoader = DataLoader(dataset=dataset,
                                     batch_size=batch_size,
                                     shuffle=True)
    def __iter__(self):
        return ((batch_X.type(torch.float), batch_y.type(torch.long))
                 for (batch_X, batch_y) in self.dataLoader)


class DataHolderGentle():
    """
    Class to store all data using the GentleLoader.

    :param config: hyper params configuration
    :type config: LRConfig or DFNConfig
    :param train_dataset: dataset of training data
    :type train_dataset: torch.utils.data.dataset.TensorDataset
    :param test_dataset: dataset of test data
    :type test_dataset: torch.utils.data.dataset.TensorDataset
    :param valid_dataset: dataset of valid data
    :type valid_dataset: torch.utils.data.dataset.TensorDataset
    :param batch_size: batch size for training
    :type test_batch: batch size for the testing data
    :param test_batch: int
    """
    def __init__(self,
                 config,
                 train_dataset,
                 valid_dataset,
                 test_dataset,
                 test_batch=1000):
        batch_size = config.batch_size
        self.train_loader = GentleLoader(dataset=train_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)
        self.valid_loader = GentleLoader(dataset=valid_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)
        self.test_loader = GentleLoader(dataset=test_dataset,
                                        batch_size=test_batch,
                                        shuffle=True)

Os diferentes dados de treinamento, teste e validação do nosso problema de carro autônomo vão ficar todos agrupados no objeto `self_driving_data`.


In [ ]:
self_driving_data = DataHolderGentle(lr_config, train_dataset, valid_dataset, test_dataset) 

Note que ao trabalharmos com imagens, nos dividimos um vetor de imagens por 255 para que o intervalo das features fique entre 0 e 1.

In [ ]:
batch_X, batch_y = next(iter(self_driving_data.train_loader))
batch_X = batch_X / 255
print("exemplo de batch de treinamento\n")
print("batch_X")
print(batch_X)
print()
print("batch_X.type =", batch_X.type())
print()
print("batch_X.shape =", batch_X.shape)
print()
print("\nbatch_y")
print(batch_y)
print()
print("batch_y.type =", batch_y.type())
print()
print("batch_y.shape =", batch_y.shape)
print()

##  Definindo um modelo 1: regressão logística

Um modo conveniente de se definir modelos em PyTorch é feito usando a classe [`torch.nn.Module`](https://pytorch.org/docs/master/_modules/torch/nn/modules/module.html#Module). Nessa classe temos que definir apenas as operações na inicialização da classe e como o modelo vai realizar o *forward pass*.

###  **Exercício 4)**

Podemos definir um modelo de regressão logística do seguinte modo:

<table>
<tr>
<td>
<img align="middle"  width='600' heith='100' src='images/logistic_regression.png'>
</td>
</tr>
</table>




Complete a classe `LogisticRegression`.

Você deve definir a inicialização dessa classe e completar os métodos `forward` e `predict`. Note que:

 - o método `forward` deve retornar apenas os *logits*.
 
 - o método `predict` deve retornar $\mathbf{\hat{y}}$.


In [ ]:
class LogisticRegression(nn.Module):
    """
    Logistic regression model.
    
    You may find nn.Linear and nn.Softmax useful here.
    
    :param config: hyper params configuration
    :type config: LRConfig
    """
    def __init__(self, config):
        super(LogisticRegression, self).__init__()
        # YOUR CODE HERE:
        raise NotImplementedError("falta completar o método __init__ da classe LogisticRegression")
        # END YOUR CODE
        

    def forward(self, x):
        """
        Computes forward pass

        :param x: input tensor
        :type x: torch.FloatTensor(shape=(batch_size, number_of_features))
        :return: logits
        :rtype: torch.FloatTensor(shape=[batch_size, number_of_classes])
        """
        # YOUR CODE HERE:
        raise NotImplementedError("falta completar o método forward da classe LogisticRegression")
        # END YOUR CODE
        return logits
    
    
    def predict(self, x):
        """
        Computes model's prediction

        :param x: input tensor
        :type x: torch.FloatTensor(shape=(batch_size, number_of_features))
        :return: model's predictions
        :rtype: torch.LongTensor(shape=[batch_size])
        """
        # YOUR CODE HERE:
        raise NotImplementedError("falta completar o método predict da classe LogisticRegression")          
        # END YOUR CODE
        return predictions        

### Testes do exercío 4

In [ ]:
lr_model = LogisticRegression(lr_config)
images = batch_X
out = lr_model(images)
assert out.type() == 'torch.FloatTensor', "problemas com o tipo da saida do método forward"
assert out.shape == torch.Size([lr_config.batch_size, lr_config.classes]), "problemas com o shape da saida do método forward"

In [ ]:
prediction = lr_model.predict(images)
assert prediction.type() == 'torch.LongTensor', "problemas com o tipo da saida do método prediction"
assert prediction.shape == torch.Size([lr_config.batch_size]), "problemas com o shape da saida do método prediction"

Antes do treinamento podemos visualizar a qualidade das predições do modelo com os parâmetros randomicamente inicializados usando uma [matriz de confusão](https://en.wikipedia.org/wiki/Confusion_matrix)

In [ ]:
img, labels = next(iter(self_driving_data.test_loader))

lr_model = LogisticRegression(lr_config)
img = img / 255
pred = lr_model.predict(img)
pred = pred.numpy()

plot_confusion_matrix(truth=labels.numpy(),
                      predictions=pred,
                      save=False,
                      path="logref_confusion_matrix.png",
                      classes=["forward", "left", "right"])

Temos os componentes principais para performar o treinamento:

- um modelo ('LogisticRegression')
- um dataset ('self_driving_data')
- um conjunto de hiper parâmetros ('LRConfig')

Agora basta criar um *loop* de treinamento. Vamos passar por todo o dataset um certo número de vezes (chamamos de *epoch* quando passamos por todo o dataset). A cada passagem vamos: 

- amostrar um batch de imagens e rótulos
- calcular a entropia cruzada entre os logits do modelo e os rótulos
- fazer o backward pass
- fazer a atualização dos pesos
- de tempos em tempos (dado pelo hiper parâmetro 'LRConfig.save_step'), vamos calcular o erro para o dataset de validação e vamos salvar os parâmetros do modelo quando vemos uma melhoria 

###  **Exercício 5)**

Você deve completar a função `train_model_img_classification`. Há 4 partes para ser completadas. Há um comentário em cada parte indicando o que deve ser feito.

In [ ]:
def train_model_img_classification(model,
                                   config,
                                   dataholder,
                                   model_path,
                                   verbose=True):
    """
    Train a model for image classification

    :param model: image classification model
    :type model: LogisticRegression or DFN
    :param config: image classification model
    :type config: LogisticRegression or DFN
    :param dataholder: data
    :type dataholder: DataHolder or DataHolderGentle
    :param model_path: path to save model params
    :type model_path: str
    :param verbose: param to control print
    :type verbose: bool
    """
    train_loader = dataholder.train_loader
    valid_loader = dataholder.valid_loader

    best_valid_loss = float("inf")
    # YOUR CODE HERE:
    # i) define the loss criteria and the optimizer. 
    # You may find nn.CrossEntropyLoss and torch.optim.SGD useful here.
    raise NotImplementedError("falta completar a função train_model_img_classification")
    # criterion =  
    # optimizer =
    # END YOUR CODE
    
    train_loss = []
    valid_loss = []
    for epoch in range(config.epochs):
        for step, (images, labels) in enumerate(train_loader):
            # YOUR CODE HERE:
            # ii) You should zero the model gradients
            # and define the loss function for the train data.
            raise NotImplementedError("falta completar a função train_model_img_classification")
            # loss = 
            # END YOUR CODE
            if step % config.save_step == 0:
                # YOUR CODE HERE:
                # iii) You should define the loss function for the valid data.
                raise NotImplementedError("falta completar a função train_model_img_classification")
                # v_loss = 
                # END YOUR CODE
                valid_loss.append(float(v_loss))
                train_loss.append(float(loss))
                if float(v_loss) < best_valid_loss:
                    msg = "\ntrain_loss = {:.3f} | valid_loss = {:.3f}".format(float(loss),float(v_loss))
                    torch.save(model.state_dict(), model_path)
                    best_valid_loss = float(v_loss)
                    if verbose:
                        print(msg, end="")
            # YOUR CODE HERE:
            # iv) You should do the back propagation
            # and do the optimization step.
            raise NotImplementedError("falta completar a função train_model_img_classification")            
            # END YOUR CODE
    if verbose:
        x = np.arange(1, len(train_loss) + 1, 1)
        fig, ax = plt.subplots(1, 1, figsize=(12, 5))
        ax.plot(x, train_loss, label='train loss')
        ax.plot(x, valid_loss, label='valid loss')
        ax.legend()
        plt.xlabel('step')
        plt.ylabel('loss')
        plt.title('Train and valid loss')
        plt.grid(True)
        plt.show()


### Testes do exercío 5

In [ ]:
accuracy_list = []

for i in range(7):
    my_lr_config = LRConfig(epochs=1)
    my_lr_model = LogisticRegression(my_lr_config)
    train_model_img_classification(my_lr_model,
                                   my_lr_config,
                                   self_driving_data,
                                   'logreg.pkl',
                                   verbose=False)
    img, labels = next(iter(self_driving_data.test_loader))
    img = img / 255
    pred = my_lr_model.predict(img)
    pred = pred.numpy()
    accuracy = np.sum(pred == labels.numpy())/ labels.shape[0]
    accuracy_list.append(accuracy)
    print("i={}, accuracy = {:.3f}".format(i, accuracy))

assert os.path.exists("logreg.pkl"), "Problemas ao salvar o modelo"
assert np.mean(accuracy_list) >= 0.6, "A acurácia média tem que ser maior que 60%"

Agora podemos treinar o modelo de regressão logística

In [ ]:
my_lr_config = LRConfig()
my_lr_model = LogisticRegression(my_lr_config)
train_model_img_classification(my_lr_model,
                               my_lr_config,
                               self_driving_data,
                               'logreg.pkl',
                               verbose=True)

Olhando a matriz de confusão com o modelo treinado

In [ ]:
img, labels = next(iter(self_driving_data.test_loader))

my_lr_model = LogisticRegression(my_lr_config)

# aqui carregamos os parametros treinados do modelo
my_lr_model.load_state_dict(torch.load('logreg.pkl'))
img = img / 255
pred = my_lr_model.predict(img)
pred = pred.numpy()

plot_confusion_matrix(truth=labels.numpy(),
                      predictions=pred,
                      save=False,
                      path="logref_confusion_matrix.png",
                      classes=["forward", "left", "right"])

Olhando alguns exemplos de classificação

In [ ]:
pred9 = pred[0:9]
pred9 = [int2command[i] for i in pred9] 
img9 = img[0:9].numpy()
labels9 = labels[0:9].numpy()
labels9 = [int2command[i] for i in labels9]
img9 = img9.reshape((9, 45, 80, 3))
img9 = img9[...,::-1]
plot9images(img9, labels9, (45, 80, 3), pred9)

##  Definindo um modelo 2: rede neural

*Deep Feedforward Networks* (DFN) são também chamadas de *feedforward neural
networks*, *multilayer perceptrons*, ou, em bom português, **redes neurais**.

Agora vamos implementar o modelo de rede neural. Como estamos estendendo a classe `nn.module` do PyTorch, definir uma rede neural não é muito mais complexo que definir um modelo de regressão logística.

Mas antes de definir o modelo, vamos construir uma classe para guardar todos os hiper parâmetros de uma rede neural.

In [ ]:
class DFNConfig(LRConfig):
    """
    Holds DFN model hyperparams.

    :param architecture: network dense architecture
    :type architecture: list of int
    """
    def __init__(self, architecture=[100, 3]):
        super(DFNConfig, self).__init__()
        self.architecture = architecture

    def __str__(self):
        """
        Get all attributs values.

        :return: all hyperparams as a string
        :rtype: str
        """
        status = "height = {}\n".format(self.height)
        status += "width = {}\n".format(self.width)
        status += "channels = {}\n".format(self.channels)
        status += "architecture = {}\n".format(self.architecture)
        status += "batch_size = {}\n".format(self.batch_size)
        status += "epochs = {}\n".format(self.epochs)
        status += "save_step = {}\n".format(self.save_step)
        status += "learning_rate = {}\n".format(self.learning_rate)
        status += "momentum = {}\n".format(self.momentum)
        return status


In [ ]:
dfn_config = DFNConfig()
print("Os hiper parâmetros do modelo de rede neural são:\n")
print(dfn_config)

## Exercicio 6: redes neurais profundas

<table>
<tr>
<td>
<img align="middle"  width='400' heith='100'  src='images/dfn.png'>
</td>
</tr>
</table>

Nesse exemplo, você vai ter que completar a classe `DFN`. 

Esssa classe instancia uma rede neural de acordo com o hiper parâmetro `config.architecture`. 

Assim, por exemplo, se `config.architecture = [200, 100, 3]`. Essa classe vai criar uma rede neural com duas camadas escondidas: uma com 200 neurônios e outra com 100 neurônios; e a camada de saída tem tamanho 3. Note que a camada de entrada deve ser criada usando `config.height`, `config.width` e `config.channels`.

Para simplificar a implementação, a não linearidade deve ser causada apenas pela função [ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks). Então, uma rede neural vai ser definida pelas fórmulas:

<table>
<tr>
<td>
<img align="middle"  width='350' heith='100'  src='images/dfn_description.png'>
</td>
</tr>
</table>





Como no caso da regressão logística, temos que:

 - o método `forward` deve retornar apenas os *logits* (os valores da última camada antes da aplicação da função softmax)
 
 - o método `predict` deve retornar $\mathbf{\hat{y}}$.

In [ ]:
class DFN(nn.Module):
    """
    Deep Feedforward Network.
    
    The method self.add_module is useful here.
    The class nn.ReLU() is useful too.
    
    :param config: hyper params configuration
    :type config: DFNConfig
    """
    def __init__(self, config):
        super(DFN, self).__init__()
        # YOUR CODE HERE:
        raise NotImplementedError("falta completar o método __init__ da classe DFN")
        # END YOUR CODE


    def forward(self, x):
        """
        Computes forward pass

        :param x: input tensor
        :type x: torch.FloatTensor(shape=(batch_size, number_of_features))
        :return: logits
        :rtype: torch.FloatTensor(shape=[batch_size, number_of_classes])
        """
        # YOUR CODE HERE:
        raise NotImplementedError("falta completar o método forward da classe DFN")
        # END YOUR CODE
        return logits
        
    
    def predict(self, x):
        """
        Computes model's prediction

        :param x: input tensor
        :type x: torch.FloatTensor(shape=(batch_size, number_of_features))
        :return: model's predictions
        :rtype: torch.LongTensor(shape=[batch_size])
        """
        # YOUR CODE HERE:
        raise NotImplementedError("falta completar o método predict da classe DFN")     
        # END YOUR CODE
        return predictions      


### Testes do exercío 6

In [ ]:
dfn_config = DFNConfig()
dfn_model = DFN(dfn_config)
images = batch_X
out = dfn_model(images)
assert out.type() == 'torch.FloatTensor', "problemas com o tipo da saida do método forward"
assert out.shape == torch.Size([dfn_config.batch_size, dfn_config.classes]), "problemas com o shape da saida do método forward"

In [ ]:
prediction = dfn_model.predict(images)
assert prediction.type() == 'torch.LongTensor', "problemas com o tipo da saida do método prediction"
assert prediction.shape == torch.Size([dfn_config.batch_size]), "problemas com o shape da saida do método prediction"

In [ ]:
def param_checker(config, model):
    """
    Check if the dfn model's has the right kind of parameters

    :param config: model's hyperparamters 
    :type config: DFNConfig
    :param model: neural network 
    :type model: DFN
    """
    input_shape = config.height * config.width * config.channels
    all_params = list(model.parameters())
    msg = "Modelo sem nenhum parâmetro"
    assert all_params != [], msg  
    architecture = [input_shape] + config.architecture
    count = 0
    for params in all_params:
        shape = tuple(params.shape)
        if len(shape) == 2:
            msgW = "{} != torch.Size([{}, {}])".format(params.shape, shape[0], shape[1]) 
            assert shape == (architecture[count + 1], architecture[count])  
        else:
            msgb = "{} != torch.Size([{}])".format(params.shape, shape[0]) 
            assert shape == (architecture[count + 1],)  
            count += 1
    print("Todos os parâmetros ok")



deep_config1 = DFNConfig(architecture=[400, 300, 200, 100, 50, 10])
deep_model1 = DFN(deep_config1)


deep_config2 = DFNConfig(architecture=[800, 600, 400, 300, 200, 100, 50, 27])
deep_model2 = DFN(deep_config2)


deep_config3 = DFNConfig(architecture=[900, 400, 300, 200, 100, 50, 13])
deep_model3 = DFN(deep_config3)

shallow_config = DFNConfig(architecture=[10])
shallow_model = DFN(shallow_config)

param_checker(deep_config1, deep_model1)
param_checker(deep_config2, deep_model2)
param_checker(deep_config3, deep_model3)
param_checker(shallow_config, shallow_model)

Vamos treinar uma rede neural com apenas duas camadas escondidas.

In [ ]:
dfn_config = DFNConfig(architecture=[200, 100, 3])
dfn_model = DFN(dfn_config)
train_model_img_classification(dfn_model,
                               dfn_config,
                               self_driving_data,
                               'dfn.pkl',
                               verbose=True)

Como no caso da regressão logística podemos inspecionar o resultados do treinamento

In [ ]:
img, labels = next(iter(self_driving_data.test_loader))

dfn_config = DFNConfig(architecture=[200, 100, 3])
dfn_model = DFN(dfn_config)


dfn_model.load_state_dict(torch.load('dfn.pkl'))
img = img / 255
pred = dfn_model.predict(img)
pred = pred.numpy()

plot_confusion_matrix(truth=labels.numpy(),
                      predictions=pred,
                      save=False,
                      path="dfn_confusion_matrix.png",
                      classes=["forward", "left", "right"])

In [ ]:
pred9 = pred[0:9]
pred9 = [int2command[i] for i in pred9] 
img9 = img[0:9].numpy()
labels9 = labels[0:9].numpy()
labels9 = [int2command[i] for i in labels9]
img9 = img9.reshape((9, 45, 80, 3))
img9 = img9[...,::-1]
plot9images(img9, labels9, (45, 80, 3), pred9)

Agora você pode treinar diferentes redes neurais com diferentes arquiteturas e ver qual é a melhor configuração para esse problema.

## Exercício 7: Competição no Kaggle

Uma vez treinado o modelo com os dados da pista de treinamento, temos que averiguar se o modelo consegue generalizar bem na pista de teste:

<table>
<tr>
<td>
<img align="middle" width='400' heith='100'  src='images/test_track.png'>
</td>
</tr>
</table>

As imagens dessa pista estão disponíveis na competição do Kaggle [MAC0460 self driving](https://www.kaggle.com/c/mac0460-self-driving). Há vários dados que não usamos da pasta 'self_driving_pi_car_data', é hora de usar todos eles! Agora você tem que usar todas as funções que você definiu até aqui e tentar treinar um modelo para a competição. Você pode usar outros modelos, outras bibliotecas e manipular os dados do jeito que achar melhor. 

Nesse último exercício você vai ter que treinar um modelo e submeter para a competição sua predição no formato [csv](https://en.wikipedia.org/wiki/Comma-separated_values). Você vai ter nota completa se você conseguir um valor maior ou igual que $0.65$ (Public Score).

A função abaixo ajuda a transformar um array de predições num csv. Boa competição!

In [ ]:
def labels2csv(labels, csv_path):
    """
    Transform an array of labels into a csv file
    to be submitted on the kaggle competition
    https://www.kaggle.com/c/mac0460-self-driving

    :param labels: labels
    :type labels: np.array
    :param csv_path: path to save csv file
    :type csv_path: str
    """
    with open(csv_path, "w") as file:
        file.write("id,label\n")
        for i, label in enumerate(labels):
            file.write("{},{}\n".format(i,label))